In [3]:
import json

In [26]:
model_output_desired = '[{"entity": "follow-up"}, {"entity": "evidence"}, {"entity": "recurrent"}, {"entity": "disease"}, {"entity": "the patient"}, {"entity": "1 year"}]'
ground_truth  =        '[{"entity": "follow-up"}, {"entity": "reality"},  {"entity": "recurrent"},  {"entity": "blood"},  {"entity": "the patient"}, {"entity": "1 year"}]'

In [36]:
class Evaluator():

    def __init__(self) -> None:
        pass
    
    def _assess_model_output(self, model_output: str) -> bool:
        good_format = True
        try :
            json.loads(model_output)
        except Exception as error:
            print(error)
            good_format = False
        return good_format

    def parse_json_offset(self, model_output: str) -> dict[str, list[str]]:
        if self._assess_model_output(model_output):
            output = json.loads(model_output)
            entities = [entity["entity"] for entity in output]
            offsets = [entity["offset"] for entity in output]
            return {"entities": entities, "offsets": offsets}
        
    def parse_json_NO_offset(self, model_output: str) -> dict[str, list[str]]:
        if self._assess_model_output(model_output):
            output = json.loads(model_output)
            entities = [entity["entity"] for entity in output]
            return {"entities": entities}
        
    def f1(self, model_output: str, ground_truth: str) -> float:
        model_output = self.parse_json_NO_offset(model_output)
        ground_truth = self.parse_json_NO_offset(ground_truth)
        model_output = model_output["entities"]
        ground_truth = ground_truth["entities"]
        TP = len(set(model_output).intersection(set(ground_truth)))
        FP = len(set(model_output).difference(set(ground_truth)))
        FN = len(set(ground_truth).difference(set(model_output)))
        # F1 = 2 * TP / (2 * TP + FN + FP)
        if TP == 0:
            return 0
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        return 2 * (precision * recall) / (precision + recall)

In [37]:
evaluator = Evaluator()
ouput = evaluator.parse_json_NO_offset(model_output_desired)
expected = evaluator.parse_json_NO_offset(ground_truth)
f1 = evaluator.f1(model_output_desired, ground_truth)
f1


0.6666666666666666